# Import Section
---

In [1]:
"""
A jupyter notebook with ipywidget helps user to train yolofastestv1.1 object detection model 
and convert to TFLite int8 and C++ model file.
"""
import shutil
import os
from subprocess import Popen
import time

import ipywidgets as widgets
from ipywidgets import interact_manual
from ipywidgets import Layout, Box, Dropdown, Label
from IPython.display import display, HTML
from IPython.display import clear_output
from ipyfilechooser import FileChooser

In [2]:
class TrainConfigAndCmdsWidgets:
    """
    train config and cmds widgets
    """
    def __init__(self):

        self.tflite_file_loc = ""
        self.datainfo_pt = None
        self.cfg_pt = None
        self.ori_weight_pt = None
        self.gen_pretrain_pt = None
        self.gen_model_pt = None
        self.gen_keras_pth = None
        self.gen_tflite_pth = None
        self.darknet_pt = os.path.join(os.getcwd(), "Yolo-Fastest-darknet", "build", "darknet", "x64", "darknet.exe")

        form_item_layout = Layout(
            display="flex",
            flex_flow="row",
            justify_content="space-between",
        )

        # create proj
        self.a_cp = widgets.Text(value="MyTask", placeholder="Type something", disabled=False)
        self.b_cp = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")
        self.c_cp = widgets.BoundedIntText(value=1, min=1, max=1000, step=1, disabled=False)
        self.d_cp = widgets.Button(description="Setting", layout=Layout(width="30%", height="30px"), button_style="success")
        self.e_cp = widgets.Text(value="..", placeholder="Type something", disabled=False)

        form_proj_items = [
            Box([Label(value="Project Name"), self.a_cp], layout=form_item_layout),
            Box([Label(value="Dataset Classes"), self.c_cp], layout=form_item_layout),
            Box([Label(value="Choose the Dataset Dir"), self.d_cp], layout=form_item_layout),
            Box([Label(value="The Chosen Dataset"), self.e_cp], layout=form_item_layout),
            Box([Label(value="Create Project Folder"), self.b_cp], layout=form_item_layout),
        ]

        self.form_create_proj = Box(
            form_proj_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 1px lightblue",
                align_items="stretch",
                width="100%",
            ),
        )

        # train
        self.a_ta = Dropdown(options=[320, 240, 224])
        self.b_ta = Dropdown(options=[320, 240, 224])
        self.c_ta = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")

        form_train_items = [
            Box([Label(value="Width"), self.a_ta], layout=form_item_layout),
            Box([Label(value="Height (Must same as Width)"), self.b_ta], layout=form_item_layout),
            Box([Label(value="Prepare the Training"), self.c_ta], layout=form_item_layout),
        ]

        self.form_output_train_cmd = Box(
            form_train_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 1px lightblue",
                align_items="stretch",
                width="100%",
            ),
        )

        # convert model cpp
        self.h_cm = Dropdown(options=["int8", "f32"])
        self.i_cm = widgets.Button(description="Convert", layout=Layout(width="30%", height="30px"), button_style="success")

        self.d_cm = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")

        form_convert_items = [
            Box([Label(value="Choose Quant Type"), self.h_cm], layout=form_item_layout),
            Box([Label(value="Convert to Tflite"), self.i_cm], layout=form_item_layout),
            Box([Label(value="Convert to Vela"), self.d_cm], layout=form_item_layout),
        ]

        self.form_output_convert_cmd = Box(
            form_convert_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                justify_content="center",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="70%",
            ),
        )

    def move_allfiles(self, src_folder, dst_folder):
        """
        move all files
        """

        files = os.listdir(src_folder)
        for f in files:
            fullpath = os.path.join(src_folder, f)
            if os.path.isdir(fullpath):  # copy whole folder
                shutil.move(fullpath, dst_folder)
                print(f"Copy finish: {f}")

    def show_headline(self, output):
        """
        show headline
        """
        html0 = widgets.HTML(value=f"<b><font color='lightblue'><font size=4>{output}</b>")
        display(html0)

    def show_main(self):
        """
        show main
        """

        intro_text = "Please Choose the setting of data prepare & train"
        html_widget = widgets.HTML(value=f"<b><font color='lightgreen'><font size=6>{intro_text}</b>")
        display(html_widget)

        # Create the accordion
        accordion = widgets.Accordion(children=[self.form_create_proj, self.form_output_train_cmd]).add_class("parentstyle")
        display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        accordion.set_title(0, "Create the Project")
        accordion.set_title(1, "Configure the Training")

        # Create a box combining with 2 elements
        box_data_train = Box(
            [accordion],
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="50%",
            ),
        )

        # Create a tab and put the 2 boxes
        tab = widgets.Tab(children=[box_data_train, self.form_output_convert_cmd]).add_class("parentstyle")
        tab_contents = ["Train", "Deployment"]
        tab.titles = tab_contents

        output_widgets = widgets.Output(layout=Layout(border="1px solid green"))

        def act_para(h_cm):

            # If any value is changed, clear the widgets
            with output_widgets:
                output_widgets.clear_output()

        # ------------------#
        # widgets.Accordion's interactive input with action function `act_para()`
        # ------------------#
        out_inter = widgets.interactive_output(act_para, {"h_cm": self.h_cm})

        display(tab, out_inter)

        # ------------------#
        # for labelimg cmd, move to outside of act_para to prevent keep trigering
        # ------------------#
        # output_widgets = widgets.Output(layout=Layout(border = '1px solid green'))
        display(output_widgets)

        def on_button_clicked_createproj(b):
            with output_widgets:
                clear_output()
                print("Create Project Folders. . .")
                self.create_dirs()

        self.b_cp.on_click(on_button_clicked_createproj)

        def on_button_clicked_choose_dataset_dir(b):
            with output_widgets:
                clear_output()
                self.choose_dataset_dir()

        self.d_cp.on_click(on_button_clicked_choose_dataset_dir)

        def on_button_clicked_prepare_train(b):
            with output_widgets:
                clear_output()
                self.prepare_train()

        self.c_ta.on_click(on_button_clicked_prepare_train)

        def on_button_clicked_convert_tflite(b):
            with output_widgets:
                clear_output()
                self.convert_tflite()

        self.i_cm.on_click(on_button_clicked_convert_tflite)

        def on_button_clicked_cpp(b):
            with output_widgets:
                clear_output()
                print("Convert to cpp & Vela. . .")
                self.convert_tflu()

        self.d_cm.on_click(on_button_clicked_cpp)

    def choose_dataset_dir(self):
        """
        choose dataset dir
        """

        path_ftflite = os.path.join(os.getcwd())
        f_dataset = FileChooser(path_ftflite)
        f_dataset.show_only_dirs = True
       
        f_dataset.title = "<b><font color='lightblue'><font size=4>Choose the directory of dataset.</b>"
        display(f_dataset)

        def act_test():
            m_src_dir = f_dataset.selected_path
            self.e_cp.value = m_src_dir
            print(f"The chosen dir: {self.e_cp.value}")
            print("Finish!")

        evt = interact_manual(act_test)
        evt.widget.children[0].description = "Choose this dataset"  # because there are 3 parameter of the evt
        evt.widget.children[0].button_style = "primary"

    def create_dirs(self):
        """
        create dirs
        """
        proj_path = os.path.join(os.getcwd(), "workspace", self.a_cp.value)
        source_path_mzoo = os.path.join(os.getcwd(), "Yolo-Fastest-darknet", "ModelZoo", "yolo-fastest-1.1_coco")
        source_path_dataset = self.e_cp.value
        if not os.path.exists(proj_path):
            os.makedirs(proj_path)
            os.makedirs(os.path.join(proj_path, "backup"))
            os.makedirs(os.path.join(proj_path, "vela"))
            # copy from ModelZoo
            shutil.copy(os.path.join(source_path_mzoo, "yolo-fastest-1.1.cfg"), proj_path)
            shutil.copy(os.path.join(source_path_mzoo, "coco.data"), proj_path)
            # copy from user's dataset
            coconame_src_pth = os.path.join(source_path_dataset, "coco.names")
            traintxt_src_pth = os.path.join(source_path_dataset, "train.txt")
            valtxt_src_pth = os.path.join(source_path_dataset, "val.txt")
            if os.path.exists(coconame_src_pth) and os.path.exists(traintxt_src_pth) and os.path.exists(valtxt_src_pth):
                shutil.copy(coconame_src_pth, proj_path)
                # update coco.data
                file1 = open(os.path.join(proj_path, "coco.data"), "r", encoding="utf-8")
                lines_list = file1.readlines()
                lines_list[0] = f"classes= {self.c_cp.value}\n"
                lines_list[1] = f"train  = {traintxt_src_pth}\n"
                lines_list[2] = f"valid  = {valtxt_src_pth}\n"
                lines_list[3] = f'names = {os.path.join(proj_path, "coco.names")}\n'
                lines_list[4] = f'backup = {os.path.join("workspace", self.a_cp.value, "backup")}\n'
                file1.close()
                file1 = open(os.path.join(proj_path, "coco.data"), "w", encoding="utf-8")
                file1.writelines(lines_list)
                file1.close()
                print("Finish!")
            else:
                print(f"There are files missing in dataset! Please check: {source_path_dataset}")
        else:
            print(f"This project is exist! : {proj_path}")

        return proj_path

    def prepare_train(self):
        """
        prepare train
        """
        proj_path = os.path.join(os.getcwd(), "workspace", self.a_cp.value)
        self.datainfo_pt = f".\workspace\\{self.a_cp.value}\coco.data"
        self.cfg_pt = f".\workspace\\{self.a_cp.value}\yolo-fastest-1.1.cfg"
        self.ori_weight_pt = ".\Yolo-Fastest-darknet\ModelZoo\yolo-fastest-1.1_coco\yolo-fastest-1.1.weights"
        self.gen_pretrain_pt = f".\workspace\\{self.a_cp.value}\yolo-fastest-1.1.conv.109"
        self.gen_model_pt = f".\workspace\\{self.a_cp.value}\\backup\yolo-fastest-1_last.weights"

        # run calc_anchors
        print("Start to run calc_anchors, please waiting for couple of minutes!")
        cmd_anchors = [self.darknet_pt, "detector", "calc_anchors", self.datainfo_pt, "-num_of_clusters", "6", "-width", str(self.a_ta.value), "-height", str(self.b_ta.value)]
        p = Popen(cmd_anchors)
        time.sleep(5.0)
        p.terminate()
        _, stderr = p.communicate()
        if stderr:
            print(stderr)
        else:
            print("calc_anchors done!")

        # update cfg
        file_anchor = open("anchors.txt", "r", encoding="utf-8")
        anchor_str = file_anchor.readlines()[0].split("\n")[0]
        file_anchor.close()

        file1 = open(os.path.join(proj_path, "yolo-fastest-1.1.cfg"), "r", encoding="utf-8")
        lines_list = file1.readlines()
        lines_list[3] = f"width={self.a_ta.value}\n"
        lines_list[4] = f"height={self.b_ta.value}\n"
        lines_list[863] = f"anchors ={anchor_str}\n"
        lines_list[931] = f"anchors ={anchor_str}\n"
        lines_list[864] = f"classes={self.c_cp.value}\n"
        lines_list[932] = f"classes={self.c_cp.value}\n"
        lines_list[857] = f"filters={(self.c_cp.value + 5) * 3}\n"
        lines_list[925] = f"filters={(self.c_cp.value + 5) * 3}\n"
        file1.close()

        file1 = open(os.path.join(proj_path, "yolo-fastest-1.1.cfg"), "w", encoding="utf-8")
        file1.writelines(lines_list)
        file1.close()
        print("update training config done!")

        # get the pre-train backbone
        cmd_partial = [self.darknet_pt, "partial", self.cfg_pt, self.ori_weight_pt, self.gen_pretrain_pt, "109"]
        print(self.darknet_pt, "partial", self.cfg_pt, self.ori_weight_pt, self.gen_pretrain_pt, "109")
        p = Popen(cmd_partial)
        _, stderr = p.communicate()
        # print(stdout)
        if stderr:
            print(stderr)
        else:
            print("partial done!")

        # get the training cmd
        self.show_headline("Please copy these commands to the CMD.exe Prompt to execute. ")

        self.show_headline("(1.) Please excute the below commands under the working directory")
        print(f"cd {os.path.join(os.getcwd())}")
        self.show_headline("(2.) Train:")
        train_cmd = rf".\Yolo-Fastest-darknet\build\darknet\x64\darknet.exe detector train {self.datainfo_pt} {self.cfg_pt} {self.gen_pretrain_pt}"
        print(train_cmd)

        # get the testing cmd
        self.show_headline("(3.) Test:")
        test_cmd = rf".\Yolo-Fastest-darknet\build\darknet\x64\darknet.exe detector map {self.datainfo_pt} {self.cfg_pt} {self.gen_model_pt} -points 0"
        print(test_cmd)

        self.show_headline("(4.) Test Single Plot:")
        test_s_cmd = rf".\Yolo-Fastest-darknet\build\darknet\x64\darknet.exe detector test {self.datainfo_pt} {self.cfg_pt} {self.gen_model_pt} -thresh 0.5"
        print(test_s_cmd)

    def convert_tflite(self):
        """
        convert tflite
        """
        darknet_tflite_convert_path = "darknet_tflite\\tools\\model_converter\\"
        self.cfg_pt = f".\\workspace\\{self.a_cp.value}\\yolo-fastest-1.1.cfg"
        self.gen_model_pt = f".\\workspace\\{self.a_cp.value}\\backup\yolo-fastest-1_last.weights"
        self.gen_keras_pth = f"workspace\\{self.a_cp.value}\\yolo-fastest-1.1.h5"
        self.gen_tflite_pth = f"workspace\\{self.a_cp.value}\\yolo-fastest-1.1-{self.h_cm.value}.tflite"
        valtxt_src_pth = os.path.join(self.e_cp.value, "val.txt")
        input_shape_size = f"{str(self.a_ta.value)}x{str(self.b_ta.value)}"

        if os.path.exists(valtxt_src_pth):
            # to keras
            python_file = darknet_tflite_convert_path + r"convert.py"
            %run $python_file $self.cfg_pt $self.gen_model_pt $self.gen_keras_pth -f

            # to tflite
            python_file = darknet_tflite_convert_path + r"post_train_quant_convert.py"
            %run $python_file --keras_model_file $self.gen_keras_pth --annotation_file $valtxt_src_pth --model_input_shape $input_shape_size --sample_num 100 --output_format $self.h_cm.value --output_file $self.gen_tflite_pth
            print(f"Finish, the output is here: {self.gen_tflite_pth}!!")
        else:
            print("Error, missing val.txt. Please enter the dataset path!!")

    def convert_tflu(self):
        """
        convert tflu
        """

        python_file = "vela\exebat.py"
        sre_dir = f"..\\workspace\\{self.a_cp.value}"
        sre_file = "yolo-fastest-1.1-int8.tflite"
        gen_dir = f"..\\workspace\\{self.a_cp.value}\\vela"
        %run $python_file --SRC_DIR $sre_dir --SRC_FILE $sre_file --GEN_DIR $gen_dir

In [3]:
act = TrainConfigAndCmdsWidgets()
act.show_main()

HTML(value="<b><font color='lightgreen'><font size=6>Please Choose the setting of data prepare & train</b>")

Output()

Output(layout=Layout(border_bottom='1px solid green', border_left='1px solid green', border_right='1px solid g…